# **About Dataset**

Cardiovascular diseases (CVDs) are the number 1 cause of death globally, taking an estimated 17.9 million lives each year, which accounts for 31% of all deaths worlwide.
Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure.

Most cardiovascular diseases can be prevented by addressing behavioural risk factors such as tobacco use, unhealthy diet and obesity, physical inactivity and harmful use of alcohol using population-wide strategies.

People with cardiovascular disease or who are at high cardiovascular risk (due to the presence of one or more risk factors such as hypertension, diabetes, hyperlipidaemia or already established disease) need early detection and management wherein a machine learning model can be of great help.

# **Variables**

* **age:** Age
* **anaemia:** Decrease of red blood cells or hemoglobin (boolean)
* **creatinine_phosphokinase:** Level of the CPK enzyme in the blood (mcg/L)
* **diabetes:** If the patient has diabetes (boolean)
* **ejection_fraction:** Percentage of blood leaving the heart at each contraction (percentage)
* **high_blood_pressure:** If the patient has hypertension (boolean)
* **platelets:** Platelets in the blood (kiloplatelets/mL)
* **serum_creatinine:** Level of serum creatinine in the blood (mg/dL)
* **serum_sodium:** Level of serum sodium in the blood (mEq/L)
* **sex:** Woman or man (binary)
* **smoking** Smoking status (boolean)

# **Import Libraries**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from sklearn import preprocessing
import statsmodels.api as sm
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler,MinMaxScaler,PowerTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
import plotly.express as ex
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')

# **Read Data and Preprocessing**

In [ ]:
main_df=pd.read_csv('/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
df=main_df.copy()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.describe().T

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(df.corr(),annot=True,linewidths=.5)

In [ ]:
df["DEATH_EVENT"].value_counts().plot.bar()

In [ ]:
df["DEATH_EVENT"].value_counts()

In [ ]:
X = df.drop(["DEATH_EVENT"],axis=1)
y = df['DEATH_EVENT']

oversample = SMOTE()
X, y = oversample.fit_resample(X, y)


df = pd.concat([pd.DataFrame(X), pd.DataFrame(y)], axis=1)
df.columns =['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time',
       'DEATH_EVENT']
df

In [ ]:
X_SMT = df["DEATH_EVENT"].value_counts().reset_index()
sns.barplot(x = "index", y = "DEATH_EVENT", data=X_SMT);

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(18, 15))
sns.boxplot(x="DEATH_EVENT",y="age",ax=axes[0,0],data=df)
sns.boxplot(x="DEATH_EVENT",y="ejection_fraction",ax=axes[0,1],data=df)
sns.boxplot(x="DEATH_EVENT",y="platelets",ax=axes[1,0],data=df)
sns.boxplot(x="DEATH_EVENT",y="serum_creatinine",ax=axes[1,1],data=df)
sns.boxplot(x="DEATH_EVENT",y="serum_sodium",ax=axes[2,0],data=df)

In [ ]:
sns.pairplot(df,hue="DEATH_EVENT")

In [ ]:
ex.pie(df,names="DEATH_EVENT",title="DEATH EVENT")

In [ ]:
df.age = df.age.astype('int64')
df.anaemia=pd.Categorical(df.anaemia)
df.diabetes=pd.Categorical(df.diabetes)
df.high_blood_pressure=pd.Categorical(df.high_blood_pressure)
df.sex=pd.Categorical(df.sex)
df.smoking=pd.Categorical(df.smoking)
df.DEATH_EVENT=pd.Categorical(df.DEATH_EVENT)

In [ ]:
df.info()

In [ ]:
plt.figure(figsize=(20,16))
ddf=df.drop('DEATH_EVENT',axis=True)
for i,column in enumerate(ddf.columns,1):
    plt.subplot(4,3,i)
    sns.distplot(ddf[column],hist=True,color="blue")
plt.show()

In [ ]:
df.columns

**In the following visualization;**

**numeric variables are taken as "True" if they are greater than the mean,**

**numeric variables are taken as "False" if they are less than the mean.**

In [ ]:
fig, axes = plt.subplots(4, 3, figsize=(20, 25))

pd.crosstab(df.age>df.age.mean() ,df.DEATH_EVENT).plot(kind='bar',ax=axes[0,0])

pd.crosstab(df.anaemia ,df.DEATH_EVENT).plot(kind='bar',ax=axes[0,1])

pd.crosstab(df.creatinine_phosphokinase>df.creatinine_phosphokinase.mean() ,df.DEATH_EVENT).plot(kind='bar',ax=axes[0,2])

pd.crosstab(df.diabetes ,df.DEATH_EVENT).plot(kind='bar',ax=axes[1,0])

pd.crosstab(df.ejection_fraction>df.ejection_fraction.mean() ,df.DEATH_EVENT).plot(kind='bar',ax=axes[1,1])

pd.crosstab(df.high_blood_pressure ,df.DEATH_EVENT).plot(kind='bar',ax=axes[1,2])

pd.crosstab(df.platelets>df.platelets.mean() ,df.DEATH_EVENT).plot(kind='bar',ax=axes[2,0])

pd.crosstab(df.serum_creatinine>df.serum_creatinine.mean() ,df.DEATH_EVENT).plot(kind='bar',ax=axes[2,1])

pd.crosstab(df.serum_sodium>df.serum_sodium.mean() ,df.DEATH_EVENT).plot(kind='bar',ax=axes[2,2])

pd.crosstab(df.smoking ,df.DEATH_EVENT).plot(kind='bar',ax=axes[3,0])

pd.crosstab(df.time>df.time.mean() ,df.DEATH_EVENT).plot(kind='bar',ax=axes[3,1])


plt.show()


In [ ]:
# numeric variables 

age_big_avg=df[df["age"]>df["age"].mean()]
age_small_avg=df[df["age"]<df["age"].mean()]
age_big_avg_survived=age_big_avg[df["DEATH_EVENT"]==0]
age_big_avg_dead=age_big_avg[df["DEATH_EVENT"]==1]
age_small_avg_survived=age_small_avg[df["DEATH_EVENT"]==0]
age_small_avg_dead=age_small_avg[df["DEATH_EVENT"]==1]

creatine_big_avg=df[df["creatinine_phosphokinase"]>df["creatinine_phosphokinase"].mean()]
creatine_small_avg=df[df["creatinine_phosphokinase"]<df["creatinine_phosphokinase"].mean()]
creatine_big_avg_survived=creatine_big_avg[df["DEATH_EVENT"]==0]
creatine_big_avg_dead=creatine_big_avg[df["DEATH_EVENT"]==1]
creatine_small_avg_survived=creatine_small_avg[df["DEATH_EVENT"]==0]
creatine_small_avg_dead=creatine_small_avg[df["DEATH_EVENT"]==1]

ejection_big_avg=df[df["ejection_fraction"]>df["ejection_fraction"].mean()]
ejection_small_avg=df[df["ejection_fraction"]<df["ejection_fraction"].mean()]
ejection_big_avg_survived=ejection_big_avg[df["DEATH_EVENT"]==0]
ejection_big_avg_dead=ejection_big_avg[df["DEATH_EVENT"]==1]
ejection_small_avg_survived=ejection_small_avg[df["DEATH_EVENT"]==0]
ejection_small_avg_dead=ejection_small_avg[df["DEATH_EVENT"]==1]

platelets_big_avg=df[df["platelets"]>df["platelets"].mean()]
platelets_small_avg=df[df["platelets"]<df["platelets"].mean()]
platelets_big_avg_survived=platelets_big_avg[df["DEATH_EVENT"]==0]
platelets_big_avg_dead=platelets_big_avg[df["DEATH_EVENT"]==1]
platelets_small_avg_survived=platelets_small_avg[df["DEATH_EVENT"]==0]
platelets_small_avg_dead=platelets_small_avg[df["DEATH_EVENT"]==1]

s_creatine_big_avg=df[df["serum_creatinine"]>df["serum_creatinine"].mean()]
s_creatine_small_avg=df[df["serum_creatinine"]<df["serum_creatinine"].mean()]
s_creatine_big_avg_survived=s_creatine_big_avg[df["DEATH_EVENT"]==0]
s_creatine_big_avg_dead=s_creatine_big_avg[df["DEATH_EVENT"]==1]
s_creatine_small_avg_survived=s_creatine_small_avg[df["DEATH_EVENT"]==0]
s_creatine_small_avg_dead=s_creatine_small_avg[df["DEATH_EVENT"]==1]

s_sodium_big_avg=df[df["serum_sodium"]>df["serum_sodium"].mean()]
s_sodium_small_avg=df[df["serum_sodium"]<df["serum_sodium"].mean()]
s_sodium_big_avg_survived=s_sodium_big_avg[df["DEATH_EVENT"]==0]
s_sodium_big_avg_dead=s_sodium_big_avg[df["DEATH_EVENT"]==1]
s_sodium_small_avg_survived=s_sodium_small_avg[df["DEATH_EVENT"]==0]
s_sodium_small_avg_dead=s_sodium_small_avg[df["DEATH_EVENT"]==1]

time_big_avg=df[df["time"]>df["time"].mean()]
time_small_avg=df[df["time"]<df["time"].mean()]
time_big_avg_survived=time_big_avg[df["DEATH_EVENT"]==0]
time_big_avg_dead=time_big_avg[df["DEATH_EVENT"]==1]
time_small_avg_survived=time_small_avg[df["DEATH_EVENT"]==0]
time_small_avg_dead=time_small_avg[df["DEATH_EVENT"]==1]

In [ ]:
# categorical variables

anaemia_yes=df[df['anaemia']==1]
anaemia_no=df[df['anaemia']==0]
anaemia_yes_survived=anaemia_yes[df["DEATH_EVENT"]==0]
anaemia_yes_dead=anaemia_yes[df["DEATH_EVENT"]==1]
anaemia_no_survived=anaemia_no[df["DEATH_EVENT"]==0]
anaemia_no_dead=anaemia_no[df["DEATH_EVENT"]==1]

diabetes_yes=df[df['diabetes']==1]
diabetes_no=df[df['diabetes']==0]
diabetes_yes_survived=diabetes_yes[df["DEATH_EVENT"]==0]
diabetes_yes_dead=diabetes_yes[df["DEATH_EVENT"]==1]
diabetes_no_survived=diabetes_no[df["DEATH_EVENT"]==0]
diabetes_no_dead=diabetes_no[df["DEATH_EVENT"]==1]

blood_pressure_yes=df[df['high_blood_pressure']==1]
blood_pressure_no=df[df['high_blood_pressure']==0]
blood_pressure_yes_survived=blood_pressure_yes[df["DEATH_EVENT"]==0]
blood_pressure_yes_dead=blood_pressure_yes[df["DEATH_EVENT"]==1]
blood_pressure_no_survived=blood_pressure_no[df["DEATH_EVENT"]==0]
blood_pressure_no_dead=blood_pressure_no[df["DEATH_EVENT"]==1]

smoking_yes=df[df['smoking']==1]
smoking_no=df[df['smoking']==0]
smoking_yes_survived=smoking_yes[df["DEATH_EVENT"]==0]
smoking_yes_dead=smoking_yes[df["DEATH_EVENT"]==1]
smoking_no_survived=smoking_no[df["DEATH_EVENT"]==0]
smoking_no_dead=smoking_no[df["DEATH_EVENT"]==1]

In [ ]:
labels=["age_big_avg_survived","age_big_avg_dead","age_small_avg_survived","age_small_avg_dead"]
values=[len(age_big_avg_survived),len(age_big_avg_dead),len(age_small_avg_survived),len(age_small_avg_dead)]
ex.pie(df,labels=labels,values=values,title="age - DEATH_EVENT",names=labels)

In [ ]:
labels=["creatine_big_avg_survived","creatine_big_avg_dead","creatine_small_avg_survived","creatine_small_avg_dead"]
values=[len(creatine_big_avg_survived),len(creatine_big_avg_dead),len(creatine_small_avg_survived),len(creatine_small_avg_dead)]
ex.pie(df,labels=labels,values=values,title="creatinine_phosphokinase - DEATH_EVENT",names=labels)

In [ ]:
labels=["ejection_big_avg_survived","ejection_big_avg_dead","ejection_small_avg_survived","ejection_small_avg_dead"]
values=[len(ejection_big_avg_survived),len(ejection_big_avg_dead),len(ejection_small_avg_survived),len(ejection_small_avg_dead)]
ex.pie(df,labels=labels,values=values,title="ejection_fraction - DEATH_EVENT",names=labels)

In [ ]:
labels=["platelets_big_avg_survived","platelets_big_avg_dead","platelets_small_avg_survived","platelets_small_avg_dead"]
values=[len(platelets_big_avg_survived),len(platelets_big_avg_dead),len(platelets_small_avg_survived),len(platelets_small_avg_dead)]
ex.pie(df,labels=labels,values=values,title="platelets - DEATH_EVENT",names=labels)

In [ ]:
labels=["s_creatine_big_avg_survived","s_creatine_big_avg_dead","s_creatine_small_avg_survived","s_creatine_small_avg_dead"]
values=[len(s_creatine_big_avg_survived),len(s_creatine_big_avg_dead),len(s_creatine_small_avg_survived),len(s_creatine_small_avg_dead)]
ex.pie(df,labels=labels,values=values,title="serum_creatine - DEATH_EVENT",names=labels)

In [ ]:
labels=["s_sodium_big_avg_survived","s_sodium_big_avg_dead","s_sodium_small_avg_survived","s_sodium_small_avg_dead"]
values=[len(s_sodium_big_avg_survived),len(s_sodium_big_avg_dead),len(s_sodium_small_avg_survived),len(s_sodium_small_avg_dead)]
ex.pie(df,labels=labels,values=values,title="serum_sodium -  DEATH_EVENT",names=labels)

In [ ]:
labels=["time_big_avg_survived","time_big_avg_dead","time_small_avg_survived","time_small_avg_dead"]
values=[len(time_big_avg_survived),len(time_big_avg_dead),len(time_small_avg_survived),len(time_small_avg_dead)]
ex.pie(df,labels=labels,values=values,title="time - DEATH_EVENT",names=labels)

In [ ]:
labels=["anaemia_yes_survived","anaemia_yes_dead","anaemia_no_survived","anaemia_no_dead"]
values=[len(anaemia_yes_survived),len(anaemia_yes_dead),len(anaemia_no_survived),len(anaemia_no_dead)]
ex.pie(df,labels=labels,values=values,title="anemia - DEATH_EVENT",names=labels)

In [ ]:
labels=["diabetes_yes_survived","diabetes_yes_dead","diabetes_no_survived","diabetes_no_dead"]
values=[len(diabetes_yes_survived),len(diabetes_yes_dead),len(diabetes_no_survived),len(diabetes_no_dead)]
ex.pie(df,labels=labels,values=values,title="diabetes - DEATH_EVENT",names=labels)

In [ ]:
labels=["blood_pressure_yes_survived","blood_pressure_yes_dead","blood_pressure_no_survived","blood_pressure_no_dead"]
values=[len(blood_pressure_yes_survived),len(blood_pressure_yes_dead),len(blood_pressure_no_survived),len(blood_pressure_no_dead)]
ex.pie(df,labels=labels,values=values,title="high_blood_pressure - DEATH_EVENT",names=labels)

In [ ]:
labels=["smoking_yes_survived","smoking_yes_dead","smoking_no_survived","smoking_no_dead"]
values=[len(smoking_yes_survived),len(smoking_yes_dead),len(smoking_no_survived),len(smoking_no_dead)]
ex.pie(df,labels=labels,values=values,title="smoking - DEATH_EVENT",names=labels)

# **Model Training and Tuning**

In [ ]:
df.columns

In [ ]:
plt.figure(figsize=(12,8))
X = df.drop(["DEATH_EVENT"],axis=1)
y = df['DEATH_EVENT']
sns.distplot(y)
plt.show()

In [ ]:
X

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)

In [ ]:
df.info()

In [ ]:
sc=StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

## **Logistic Regression**

In [ ]:
lr=LogisticRegression(solver="liblinear")
lr_model=lr.fit(X_train,y_train)
lr_model.intercept_

In [ ]:
y_pred=lr_model.predict(X_test)
confusion_matrix(y_test,y_pred)

In [ ]:
accuracy_score(y_test,y_pred) # primitive score

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
logit_roc_auc= roc_auc_score(y_test,lr_model.predict(X_test))
fpr,tpr,thresholds = roc_curve(y_test,lr_model.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr,tpr,label="AUC (area = %0.2f)" % logit_roc_auc)
plt.plot([0,1],[0,1],"r--")
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title("ROC")
plt.show()

### Logistic Regression Tuning

In [ ]:
lr=LogisticRegression(random_state=42)
lr_tuned=lr.fit(X_train,y_train)
accuracy_score(y_test,lr_tuned.predict(X_test))

## **Gaussian Naive Bayes**

In [ ]:
nb=GaussianNB()
nb_model=nb.fit(X_train,y_train)
nb_model.predict(X_test)[0:10]

In [ ]:
nb_model.predict_proba(X_test)[0:10]

In [ ]:
y_pred=nb_model.predict(X_test)
accuracy_score(y_test,y_pred)

### Gaussian Naive Bayes Tuning

In [ ]:
nb_params={'var_smoothing': np.logspace(0,-9, num=100)}
nb_cv=GridSearchCV(estimator=nb, 
                 param_grid=nb_params, 
                 cv=10, 
                 verbose=1, 
                 scoring='accuracy') 
nb_cv.fit(X_train,y_train)
nb_cv.best_params_

In [ ]:
nb=GaussianNB(**nb_cv.best_params_)
nb_tuned=nb.fit(X_train,y_train)

y_pred=nb_tuned.predict(X_test)
accuracy_score(y_test,y_pred)

## **K-Nearest Neighbors (KNN)**

In [ ]:
knn=KNeighborsClassifier()
knn_model=knn.fit(X_train,y_train)
y_pred=knn_model.predict(X_test) 
accuracy_score(y_test,y_pred)

In [ ]:
print(classification_report(y_test,y_pred))

### KNN Tuning

In [ ]:
knn_params={"n_neighbors":np.arange(1,50)}
knn=KNeighborsClassifier()
knn_cv=GridSearchCV(knn,knn_params,cv=10).fit(X_train,y_train)
knn_cv.best_params_

In [ ]:
knn=KNeighborsClassifier(**knn_cv.best_params_)
knn_tuned=knn.fit(X_train,y_train)
y_pred=knn_tuned.predict(X_test)
accuracy_score(y_test,y_pred)

## **Support Vector Machines (SVM)**

In [ ]:
svm_model=SVC(random_state=42).fit(X_train,y_train)
y_pred=svm_model.predict(X_test)
accuracy_score(y_test,y_pred)

### SVM Tuning

In [ ]:
svc_params={"kernel" : ["rbf"],
                 "gamma": [0.001, 0.01, 0.1, 1],
                 "C": [1,10,50,100,200,300,1000]}

svc=SVC(random_state=42)
svc_cv_model=GridSearchCV(svc,svc_params,cv=10,n_jobs=-1,verbose=1).fit(X_train,y_train)
svc_cv_model.best_params_

In [ ]:
svc_tuned=SVC(**svc_cv_model.best_params_).fit(X_train,y_train)
y_pred=svc_tuned.predict(X_test)
accuracy_score(y_test,y_pred)

## **Artificial Neural Networks (ANN)**

In [ ]:
scaler=StandardScaler()
scaler.fit(X_train)
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)
mlpc=MLPClassifier(random_state=42).fit(X_train_scaled,y_train)

y_pred=mlpc.predict(X_test_scaled)
accuracy_score(y_test,y_pred)

### ANN Tuning

In [ ]:
mlpc_params={"alpha":[0.1,0.01,0.02,0.005,0.0001,0.00001],
            "hidden_layer_sizes":[(10,10,10),(100,100,100),(100,100),(3,5),(5,3)],
            "solver":["lbfgs","adam","sgd"],
            "activation":["relu","logistic"],
            "random_state":[42]}

mlpc=MLPClassifier(random_state=42)
mlpc_cv_model=GridSearchCV(mlpc,mlpc_params,cv=10,n_jobs=-1,verbose=2).fit(X_train_scaled,y_train)
mlpc_cv_model.best_params_

In [ ]:
mlpc_tuned=MLPClassifier(**mlpc_cv_model.best_params_).fit(X_train,y_train)
y_pred=mlpc_tuned.predict(X_test)
accuracy_score(y_test,y_pred)

## **Classification and Regression Trees (CART)**

In [ ]:
cart=DecisionTreeClassifier(random_state=42)
cart_model=cart.fit(X_train,y_train)

y_pred=cart_model.predict(X_test)
accuracy_score(y_test,y_pred)

### CART Tuning

In [ ]:
cart_params={"max_depth":list(range(1,10)),
          "min_samples_split":list(range(2,50)),}

cart=tree.DecisionTreeClassifier(random_state=42)
cart_cv_model=GridSearchCV(cart,cart_params,cv=10,n_jobs=-1,verbose=2).fit(X_train,y_train)
cart_cv_model.best_params_

In [ ]:
cart=tree.DecisionTreeClassifier(**cart_cv_model.best_params_)
cart_tuned=cart.fit(X_train,y_train)

y_pred=cart_tuned.predict(X_test)
accuracy_score(y_test,y_pred)

## **Random Forests (RF)**

In [ ]:
rf_model=RandomForestClassifier(random_state=42).fit(X_train,y_train)
y_pred=rf_model.predict(X_test)
accuracy_score(y_test,y_pred)

### RF Tuning

In [ ]:
rf_params={"max_depth":[2,5,8,10],
          "max_features":[2,5,8],
          "n_estimators":[10,500,1000],
          "min_samples_split":[2,5,10]}


rf_model=RandomForestClassifier(random_state=42)
rf_cv_model=GridSearchCV(rf_model,rf_params,cv=10,n_jobs=-1,verbose=2).fit(X_train,y_train)

rf_cv_model.best_params_

In [ ]:
rf_tuned=RandomForestClassifier(**rf_cv_model.best_params_).fit(X_train,y_train)
y_pred=rf_tuned.predict(X_test)
accuracy_score(y_test,y_pred)

## **Gradient Boosting Machines (GBM)**

In [ ]:
gbm_model=GradientBoostingClassifier(random_state=42).fit(X_train,y_train)
y_pred=gbm_model.predict(X_test)
accuracy_score(y_test,y_pred)

### GBM Tuning

In [ ]:
gbm_params={"n_estimators":[5,50,250,500,1000],
            "max_depth":[1,3,5,7,9],
            "learning_rate":[0.01,0.1,1,2]}

gbm=GradientBoostingClassifier(random_state=42)
gbm_cv_model=GridSearchCV(gbm,gbm_params,cv=10,n_jobs=-1,verbose=2).fit(X_train,y_train)

gbm_cv_model.best_params_

In [ ]:
gbm_tuned=GradientBoostingClassifier(**gbm_cv_model.best_params_).fit(X_train,y_train)
y_pred=gbm_tuned.predict(X_test)
accuracy_score(y_test,y_pred)

## **eXtreme Gradient Boosting (XGBM)**

In [ ]:
xgb_model=XGBClassifier(random_state=42).fit(X_train,y_train)
y_pred=xgb_model.predict(X_test)
accuracy_score(y_test,y_pred)

### XGBM Tuning

In [ ]:
"""
xgb_params={"n_estimators":[500,1000],
           "max_depth":[4,5,6],
           "learning_rate":[0.01,0.1],
           "tree_method":["hist"],
           "max_delta_step":[1]}

xgb=XGBClassifier(random_state=42)
xgb_cv_model=GridSearchCV(xgb,xgb_params,cv=10,n_jobs=-1,verbose=2).fit(X_train,y_train)

xgb_cv_model.best_params_
"""

In [ ]:
xgb_tuned=XGBClassifier(random_state=42,learning_rate=0.001,max_depth=7,n_estimators=1500).fit(X_train,y_train)

y_pred=xgb_tuned.predict(X_test)
accuracy_score(y_test,y_pred)

## **LightGBM**

In [ ]:
lgbm_model=LGBMClassifier().fit(X_train,y_train)
y_pred=lgbm_model.predict(X_test)
accuracy_score(y_test,y_pred)

### LightGBM Tuning

In [ ]:
lgbm_params={"n_estimators":[100,500,1000],
           "max_depth":[3,4,5,6],
           "learning_rate":[0.1,0.01,0.02,0.05],
           "min_child_samples":[2,5,10,20]}

lgbm=LGBMClassifier()
lgbm_cv_model=GridSearchCV(lgbm,lgbm_params,cv=10,n_jobs=-1,verbose=2).fit(X_train,y_train)
lgbm_cv_model.best_params_

In [ ]:
lgbm_tuned=LGBMClassifier(**lgbm_cv_model.best_params_).fit(X_train,y_train)
y_pred=lgbm_tuned.predict(X_test)
accuracy_score(y_test,y_pred)

## **CatBoost**

In [ ]:
catb_model=CatBoostClassifier(random_state=42).fit(X_train,y_train)

y_pred=catb_model.predict(X_test)

In [ ]:
accuracy_score(y_test,y_pred)

### CatBoost Tuning

In [ ]:
catb_params={"iterations":[1000,1500],
            "learning_rate":[0.01,0.05,0.1],
            "depth":[3,5,8],
            "random_state":[42]}
catb=CatBoostClassifier(random_state=42)
catb_cv_model=GridSearchCV(catb,catb_params,cv=10,n_jobs=-1,verbose=2).fit(X_train,y_train)

catb_cv_model.best_params_

In [ ]:
catb_tuned=CatBoostClassifier(**catb_cv_model.best_params_).fit(X_train,y_train)

y_pred=catb_tuned.predict(X_test)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
models=[knn_tuned,
         lr_tuned,
         svc_tuned,
         nb_tuned,
         mlpc_tuned,
         cart_tuned,
         rf_tuned,
         gbm_tuned,
         catb_tuned,
         lgbm_model,
         xgb_tuned]


result=[]

results=pd.DataFrame(columns=["Models","Accuracy"])

for model in models:
    names=model.__class__.__name__
    y_pred=model.predict(X_test)
    accuracy=accuracy_score(y_test,y_pred)
    result=pd.DataFrame([[names,accuracy*100]],columns=["Models","Accuracy"])
    results=results.append(result)
    
results = results.sort_values('Accuracy').reset_index()

sns.barplot(x="Accuracy",y="Models",data=results,color="r")

In [ ]:
del results["index"]
results